In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
import csv

In [2]:
#es necesario limpiar el archivo de tweets para que no se sesgue la red neuronal con las
#etiquetas de ironía y sarcasmo
infile = "ironicos_reduced.txt"
#agregaremos los tweets etiquetados a un archivo
outfile = "tweets_limpios.txt"

delete_list = ["#ironía", "#ironia", "#sarcasmo", "sarcasmo",","]
fin = open(infile)
fout = open(outfile, "w+")
for line in fin:
    for word in delete_list:        
        line = line.lower().replace(word, "")          
        line = line.replace("\n", ", 1\n")  
        line = line.replace("1 1 1 1", "")
    fout.write(line)
fin.close()
print "Se limpió el dataset de tweets ironicos y se agregó etiqueta"

Se limpió el dataset de tweets ironicos y se agregó etiqueta


In [3]:
#agregamos al dataset el conjunto de tweets no irónicos etiquetados
infile = "no_ironicos_reduced.txt"

fin = open(infile)
for line in fin:
    for word in delete_list:        
        line = line.lower().replace(",", "")          
        line = line.replace("\n", ", 0\n")   
        line = line.replace("0 0 0 0", "")
    fout.write(line)
fin.close()
fout.close()
print "Se limpió el dataset de tweets no ironicos y se agregó etiqueta"

Se limpió el dataset de tweets no ironicos y se agregó etiqueta


In [4]:
#convertimos a csv
with open('tweets_limpios.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('tweets.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('tweet', '1'))
        writer.writerows(lines)

In [5]:
#Checamos el encabezado de los datos, 1 representa que es ironico, 0 que no lo es
tweets = pd.read_csv('tweets.csv', names = ["tweet", "1"])
tweets.head

<bound method DataFrame.head of                                                   tweet  1
0                                                 tweet  1
1                                 se nota el interés.    1
2            este año he aprendido mucho de inglés...    1
3     uy trump! luego de la fuga del #chapo estoy to...  1
4     que fresquito mas bueno hace en la calle no?  ...  1
5     hace calor abro la ventana y no hay anda mejor...  1
6     gracias por dejarme sin internet @tigo_costari...  1
7          en el anuario de este año se han lucido...    1
8                           que bien empezamos el dia    1
9     para nada hay corrupción y negocios ilícitos e...  1
10    es increíble lo rapidísimo que me van los dato...  1
11    te confío una vida pero no el dinero 😕 😓 😨�...  1
12               españa país organizado y trabajador.    1
13    @enclmdiario como va a ser cierto esto si @mdc...  1
14    creo que en el grupo de nosotros no podemos se...  1
15    cuando estas en esos 

In [6]:
#descripción de los datos
tweets.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
1,6001.0,0.500083,0.500042,0.0,0.0,1.0,1.0,1.0


In [7]:
tweets.shape

(6001, 2)

In [8]:


y = tweets.drop("tweet",axis=1)
X = tweets["tweet"]

#separamos los datos para entrenamiento y experimento
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
#tamaño del dataset de entrenamiento
X_train.shape

(4500,)

In [10]:
#tamaño del dataset de prueba
X_test.shape

(1501,)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
fsw = file("spanish_stop_words.txt")
stop_w = fsw.read().splitlines()
vectorizer = TfidfVectorizer(stop_words = fsw)
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.fit_transform(X_test).todense()

#Fit de los datos de entrenamiento
#Primero escalamos los datos
scaler = StandardScaler()
scaler.fit(X_train)
StandardScaler(copy=False , with_mean=False, with_std=False)

#Aplicamos la escala a la transformación de los datos:

X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

In [13]:
#generamos el modelo de perceptron multicapas 
#pasamos como número máximo de iteraciones y 5 capas en cada neurona
mlp = MLPClassifier(hidden_layer_sizes=(5,5,5),max_iter=150)
mlp.fit(X_train, y_train)


/Users/karlagarcia/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 5, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=150, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [15]:
#realizamos las predicciones sobre los tweets de prueba
predictions = mlp.predict(X_train)
print(confusion_matrix(y_train,predictions))

[[2215    0]
 [  11 2274]]


In [16]:
outfile = "reporte.txt"
fout = open(outfile, "w+")
fout.write(classification_report(y_train, predictions))
fout.close()

In [46]:
len(mlp.coefs_)

4

In [47]:
len(mlp.coefs_[0])

1

In [48]:
len(mlp.intercepts_[0])

2